In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import seaborn as sns
import matplotlib.pyplot as plt

---
# 1. 不均衡データとは

> クラスに属するサンプル数に偏りがあるデータを不均衡データという

# 2. 不均衡データの問題点
> 不均衡データで学習させた場合、すべてのデータを負例と判定したり、再現率/適合率が低くなり結果F値も低くなったりする。

In [2]:
HR_DATASET_PATH = 'datasets/HR_comma_sep.csv'
hr_df = pd.read_csv(HR_DATASET_PATH)
hr_df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
hr_df.left.value_counts()

0    11428
1     3571
dtype: int64

In [12]:
from sklearn.svm import LinearSVC # 線形SVM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

use_cols = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
    'Work_accident',
    'promotion_last_5years',
]

# 離職者数:在籍者数 = 3571:11428の不均衡データ
X = hr_df[use_cols]
Y = hr_df.left

# 標準化
transformed_cols = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
]
ss = StandardScaler()
ss.fit(X[transformed_cols])
X[transformed_cols] = ss.transform(X[transformed_cols])


# 交差検証(ホールドアウト法)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, train_size=0.7, random_state=0)

linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)

print('学習データ数: %s' % x_train.shape[0])
print('学習データのうち離職した人数: %s' % y_train[y_train == 1].shape[0])
print('検証データ数: %s' % x_test.shape[0])
print('検証データのうち離職した人数: %s' % y_test[y_test == 1].shape[0])

y_pred = linear_svc.predict(x_test)

print('---モデルの評価---')
print('正確度: %s' % accuracy_score(y_test, y_pred))
print('適合率: %s' % precision_score(y_test, y_pred))
print('再現率: %s' % recall_score(y_test, y_pred))
print('F値: %s' % f1_score(y_test, y_pred))

print('---分割表---')
confusion_df = pd.DataFrame(confusion_matrix(y_test, y_pred), index=['在籍者', '離職者'], columns=['在籍すると予測', '離職すると予測'])
confusion_df

/Users/taiyou/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/taiyou/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


学習データ数: 10499
学習データのうち離職した人数: 2533
検証データ数: 4500
検証データのうち離職した人数: 1038
---モデルの評価---
正確度: 0.771555555556
適合率: 0.509842519685
再現率: 0.249518304432
F値: 0.335058214748
---分割表---


,在籍すると予測,離職すると予測
在籍者,3213,249
離職者,779,259


---
# 3. 対処方法

- **algorithm-level approaches**: 不均衡を調整する係数をモデルに導入する（コスト関数を調整する）
- **data-level approaches**: 正例と負例のサンプル数を調整する
    - **アンダーサンプリング**: 多数派データを減少させる
    - **オーバーサンプリング**: 少数派データを増加させる
    - **ハイブリッド法**: アンダーサンプリングとオーバーサンプリングの両方を行う
    
以下の具体的なコードでは、`imbalanced-learn`パッケージを用いる。
```
pip install -U imbalanced-learn
```

---
## チューニング(algorithm-level approaches)

クラス重み付けを調整してサンプルサイズが小さい方のクラスの影響力を上げてやる。

上記の通り、不均衡データだとサンプルサイズが大きい方のクラス（正例の方が多ければ正方向に、負例の方が多ければ負方向に）に引っ張られてしまうわけです。例えば、学習の際に負例の方が多ければ、検証するときには「全部負」という分類結果になってしまう。

そこで、例えばSVMであればマージンのところに正例と負例とで異なる重み付けをかけることでペナルティの量を変える→クラス分類結果にも意図的にバイアスをかけて正例の検出感度を上げる、みたいなことをしてやるというのがここで提案されている解決策です。

Pythonのsklearn.svm.SVCならclass_weight引数に、それぞれ正例と負例にかける重み付けの値を指定して入れてやるだけです。

値の目安ですが、基本的には2つのクラスのサンプルサイズ比を取り、多い方のクラスを1に固定して少ない方のクラスにはその比率をかける、というのが一般的なようです。それだけ少ない方のクラスに強めの影響力を持たせてやる、という感じですかね。

参考サイト

 - [SVM: Separating hyperplane for unbalanced classes — scikit-learn 0.19.0 documentation](http://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane_unbalanced.html)
 - [不均衡データをSVMでクラス分類するにはどうすれば良いか - 六本木で働くデータサイエンティストのブログ](http://tjo.hatenablog.com/entry/2014/10/09/224106)

```python
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix


x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, train_size=0.7, random_state=0)

# ※leftが0が多い!
CLASS_WEIGHT = {1: len(Y[Y == 1])/len(Y[Y == 0])}
linear_svc = LinearSVC(class_weight=CLASS_WEIGHT)
linear_svc.fit(x_train, y_train)

print('学習データ数: %s' % x_train.shape[0])
print('学習データのうち離職した人数: %s' % y_train[y_train == 1].shape[0])
print('検証データ数: %s' % x_test.shape[0])
print('検証データのうち離職した人数: %s' % y_test[y_test == 1].shape[0])

y_pred = linear_svc.predict(x_test)

print('---モデルの評価---')
print('正確度: %s' % accuracy_score(y_test, y_pred))
print('適合率: %s' % precision_score(y_test, y_pred))
print('再現率: %s' % recall_score(y_test, y_pred))
print('F値: %s' % f1_score(y_test, y_pred))

print('---分割表---')
confusion_df = pd.DataFrame(confusion_matrix(y_test, y_pred), index=['在籍者', '離職者'], columns=['在籍すると予測', '離職すると予測'])
confusion_df
```

---
## アンダーサンプリング: 多数派データを減少させる

 - 参考コード: https://github.com/scikit-learn-contrib/imbalanced-learn/tree/master/examples/under-sampling

### 1. 多数派データをランダムに抽出する
 - **<font color="blue">メリット</font>**: `sample()`を使うだけ簡単に実装できる
 - **<font color="red">デメリット</font>**: 元の多数派データの特徴量の分布が崩れてしまう可能性がある

```python
X1 = hr_df[hr_df.left == 1][use_cols]
X0 = hr_df[hr_df.left == 0][use_cols].sample(len(X1))
X = pd.concat([X1, X0])
Y = hr_df.loc[X.index, 'left']
```
もしくは、
```python
from imblearn.under_sampling import RandomUnderSampler

X = hr_df[use_cols]
Y = hr_df.left

# Apply the random under-sampling
rus = RandomUnderSampler(return_indices=True, random_state=0)
x_resampled, y_resampled, idx_resampled = rus.fit_sample(X, Y)
```

RandomUnderSamplerの引数
```python
RandomUnderSampler(
         ratio='auto',
         return_indices=False,
         random_state=None,
         replacement=False
     )
```
 - ratio : str, dict, or callable, optional (default='auto'). Ratio to use for resampling the data set.
 - return_indices : bool, optional (default=False). Whether or not to return the indices of the samples randomly selected from the majority class.
 - random_state : int, RandomState instance or None, optional (default=None)
 - replacement : boolean, optional (default=False). Whether the sample is with or without replacement.

### 2. Extraction of majority-minority Tomek links
```python
from imblearn.under_sampling import TomekLinks

# remove Tomek links
tl = TomekLinks(return_indices=True)
X_resampled, y_resampled, idx_resampled = tl.fit_sample(X, Y)
```

### 3. Under-sampling with Cluster Centroids
```python

from imblearn.under_sampling import ClusterCentroids

# Apply Cluster Centroids
cc = ClusterCentroids()
X_resampled, y_resampled = cc.fit_sample(X, Y)

# Use hard voting instead of soft voting
cc = ClusterCentroids(voting='hard')
X_resampled, y_resampled = cc.fit_sample(X, Y)
```

### 4. NearMiss-(1 & 2 & 3)
### 5. Condensend Nearest Neighbour
### 6. One-Sided Selection
### 7. Neighboorhood Cleaning Rule
```python
from imblearn.under_sampling import NeighbourhoodCleaningRule

ncr = NeighbourhoodCleaningRule(random_state=0)
X_res, y_res = ncr.fit_sample(X, Y)
```

### 8. Edited Nearest Neighbours
### 9. Instance Hardness Threshold
### 10. Repeated Edited Nearest Neighbours
### 11. AllKNN

---
## オーバーサンプリング: 少数派データを増加させる

 - 参考コード: https://github.com/scikit-learn-contrib/imbalanced-learn/tree/master/examples/over-sampling

### 1. 少数派データの複製(Random minority over-sampling with replacement)
 - **<font color="blue">メリット</font>**: ゲロ簡単
 - **<font color="red">デメリット</font>**: 過学習を引き起こしやすくなる

```python
from imblearn.over_sampling import RandomOverSampler

# Apply the random over-sampling
ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_sample(X, Y)
```

### 2. SMOTE - Synthetic Minority Over-sampling Technique

データセットにおけるそれぞれのデータの k 最近傍を基に新たなデータを生成する。

 - https://www.jair.org/media/953/live-953-2037-jair.pdf
 - http://qiita.com/shima_x/items/370587304ef17e7a61b8#over-sampling

**SMOTEのアルゴリズム**$(T, N, k)$

入力: 
 - $T=$少数派クラスのサンプル数(Number of minority class samples)
 - $N=$SMOTEの量(Amount of SMOTE)
 - $k=$最近傍数(Number of nearest neighbors)

出力: $(N/100) \times T$ 合成した少数派クラスのサンプル数

(∗ If N is less than 100%, randomize the minority class samples as only a random　percent of them will be SMOTEd. ∗)
$
if\quad N<100\\ 
\qquad then\quad 少数派データをランダム化(Randomize\quad the\quad T\quad minority\quad class\quad samples)\\ \qquad T=(N/100)\ast T\\
\qquad N=100\\ 
endif
$
$
\\ 
N=(int)(N/100)(∗The\quad amount\quad of\quad SMOTE\quad is\quad assumed\quad to\quad be\quad inintegral\quad multiples\quad of\quad 100.∗)\\ 
k=Number\quad of\quad nearest\quad neighbors\\ 
numattrs=Number\quad of\quad attributes\\ 
Sample[][]:array\quad for\quad original\quad minority\quad class\quad samples\\ 
newindex:keeps\quad a\quad count\quad of\quad number\quad of\quad synthetic\quad samples\quad generated,\quad initialized\quad to\quad 0\\ 
Synthetic[][]:array\quad for\quad synthetic\quad samples
$

少数派データのサンプルごとにk最近傍を計算(Compute k nearest neighbors for each minority class sample only)

$
for\quad i←1toT\\ 
\qquad Compute\quad k\quad nearestneighbors\quad for\quad i,\quad and\quad save\quad the\quad indices\quad in\quad the\quad nnarray\\
\qquad Populate(N,i,nnarray)\\ 
endfor
$

Populate(N, i, nnarray) (∗ Function to generate the synthetic samples. ∗)

$
while\quad N\neq 0\\ 
\qquad 1からkからランダムな値を選び,その値をnnとする.\\ 
\qquad このステップはiデータのk最近傍データ(k\quad nearest\quad neighbors\quad of\quad i)のうち一つだけ選ぶ\\ 
\qquad for\quad attr←1\quad to\quad numattrs\\ 
\qquad \qquad Compute:dif=Sample[nnarray[nn]][attr]-Sample[i][attr]\\ 
\qquad \qquad Compute:gap=random\quad number\quad between\quad 0\quad and\quad 1\\ 
\qquad \qquad Synthetic[newindex][attr]=Sample[i][attr]+gap∗dif\\ 
\qquad endfor\\ 
\qquad newindex++\\ 
\qquad N=N−1\\ 
endwhile\\ 
return(∗EndofPopulate.∗)
$

 - **<font color="blue">メリット</font>**: 
 - **<font color="red">デメリット</font>**: 

```python
from imblearn.over_sampling import SMOTE

# Apply regular SMOTE
sm = SMOTE(kind='regular')
X_res, y_res = sm.fit_sample(X, Y)
```

### 3. bSMOTE(1 & 2) - Borderline SMOTE of types 1 and 2

```python
from imblearn.over_sampling import SMOTE

sm = SMOTE(kind='borderline1')
X_res, y_res = sm.fit_sample(X, Y)
```
or
```python
from imblearn.over_sampling import SMOTE

sm = SMOTE(kind='borderline2')
X_res, y_res = sm.fit_sample(X, Y)
```

### 4. SVM SMOTE - Support Vectors SMOTE
```python
from imblearn.over_sampling import SMOTE

sm = SMOTE(kind='svm')
X_res, y_res = sm.fit_sample(X, Y)
```

### 5. ADASYN - Adaptive synthetic sampling approach for imbalanced learning

```python
from imblearn.over_sampling import ADASYN

# default options
ada = ADASYN(ratio='auto',
                 random_state=None,
                 n_neighbors=5,
                 n_jobs=1)
X_res, y_res = ada.fit_sample(X, Y)
```

---
# 参考文献
 - [不均衡データのクラス分類](https://www.slideshare.net/sfchaos/ss-11307051)
 - [不均衡データにおけるsampling - Qiita](http://qiita.com/shima_x/items/370587304ef17e7a61b8)
 - [不均衡データに対するClassification - Qiita](http://qiita.com/ryouta0506/items/619d9ac0d80f8c0aed92)
 - [Welcome to imbalanced-learn documentation! — imbalanced-learn 0.3.0 documentation](http://contrib.scikit-learn.org/imbalanced-learn/stable/)
 - [scikit-learn-contrib/imbalanced-learn: Python module to perform under sampling and over sampling with various techniques.](https://github.com/scikit-learn-contrib/imbalanced-learn)